In [2]:
import random
import numpy as np
import math

In [3]:
# time: 13:26
# stoping: 14:00
# end stop: 15:00
# end: 15:10
# total : 44m

In [10]:
def genData(n,l):
    rotL = lambda x: x[1:] + [x[0]]
    rand = lambda x: map(lambda y: random.choice([0,1]), range(l))
    
    x = map(rand, range(n))
    y = map(rotL, x)
    
    x = map(np.array, x)
    y = map(np.array, y)
    
    return zip(x,y)

genData(5,2)

[(array([0, 0]), array([0, 0])),
 (array([1, 1]), array([1, 1])),
 (array([1, 0]), array([0, 1])),
 (array([0, 1]), array([1, 0])),
 (array([1, 0]), array([0, 1]))]

In [31]:
np.set_printoptions(precision = 2)

def sigmoid(z):
    return 1.0/(1 + math.exp(-z))

def dsigmoid(z):
    return (1 - sigmoid(z))*sigmoid(z)
    
class NN:
    def __init__(self, ll, lr = 0.05):
        self.ll   = ll
        self.lr   = lr        
        self.fA   = np.vectorize(sigmoid)
        self.dfA  = np.vectorize(dsigmoid)
        self.w, self.b = self.genWB()
    
    def genWB(self):
        w = [np.random.randn(nin,nout) for (nin,nout) in zip(self.ll[:-1], self.ll[1:])]
        b = [np.random.randn(nout)     for nout     in self.ll[1:] ]
        
        return w,b
    
    def ff(self,x):        
        v  = [None for l in self.ll]
        vf = [None for l in self.ll]
        
        v [0] =   x
        vf[0] = self.fA(x)                
        
        for i in range(len(self.w)):
            v[i+1]  = np.dot(np.transpose(self.w[i]),v[i]) + self.b[i]
            vf[i+1] = self.fA(v[i+1])
            
        return v,vf
    
    def ff_pure(self,x):
        v = x
        for i in range(len(self.w)):
            v  = np.dot(np.transpose(self.w[i]),v) + self.b[i]
            
        return self.fA(v)
    
    def fb(self,v,vf,y):
        g  = [None for i in self.w]
        gw = [None for i in self.w]
        
        g[-1] = (vf[-1] - y)*self.dfA(v[-1])
        for i in range(len(self.w) -2,-1,-1):
            g[i] = np.multiply(np.dot(self.w[i+1], g[i+1]), self.dfA(v[i+1]))
        
        for i in range(len(self.w) -1,-1,-1):
            gw[i] = np.transpose(np.outer(g[i], v[i]))
            
        return gw, g
    
    def trainStep(self,x,y):
        v,vf  = self.ff(x)
        gw,gb = self.fb(v,vf,y)
        
        self.w = self.w - np.multiply(gw, self.lr)
        self.b = self.b - np.multiply(gb, self.lr)
        
        vf2   = self.ff_pure(x)
        [e1,e2] = self.err(vf[-1], y), self.err(vf2,y)
        return [e1,e2]
        
    def err(self, out, y):
        return np.float(math.sqrt(sum((out - y) ** 2)))
        
    def train(self, data):
        errs = []
        for (x,y) in data:
            errs += [self.trainStep(x,y)]
        
    
    def test(self,data):
        for (x,y) in data:
            out = self.ff_pure(x)
            err = self.err(out,y)
            print "x = {},  y = {}, out = {}, e = {}".format(x,y,out,err)

dat = genData(5000,2)
layers = [[2,3,2],[2,2,2]]

for ll in layers:
    print "ll = {}".format(ll)
    nn = NN(ll)
    nn.train(dat)
    nn.test(dat[:10])
        

ll = [2, 3, 2]
x = [1 0],  y = [0 1], out = [ 0.05  0.96], e = 0.0625973136266
x = [0 1],  y = [1 0], out = [ 0.96  0.04], e = 0.0577703900857
x = [1 1],  y = [1 1], out = [ 0.97  0.97], e = 0.0449115851178
x = [1 1],  y = [1 1], out = [ 0.97  0.97], e = 0.0449115851178
x = [1 1],  y = [1 1], out = [ 0.97  0.97], e = 0.0449115851178
x = [0 1],  y = [1 0], out = [ 0.96  0.04], e = 0.0577703900857
x = [1 0],  y = [0 1], out = [ 0.05  0.96], e = 0.0625973136266
x = [0 0],  y = [0 0], out = [ 0.03  0.03], e = 0.0463306971331
x = [0 0],  y = [0 0], out = [ 0.03  0.03], e = 0.0463306971331
x = [1 1],  y = [1 1], out = [ 0.97  0.97], e = 0.0449115851178
ll = [2, 2, 2]
x = [1 0],  y = [0 1], out = [ 0.05  0.95], e = 0.0761628654009
x = [0 1],  y = [1 0], out = [ 0.95  0.06], e = 0.079368162222
x = [1 1],  y = [1 1], out = [ 0.94  0.95], e = 0.0809375650226
x = [1 1],  y = [1 1], out = [ 0.94  0.95], e = 0.0809375650226
x = [1 1],  y = [1 1], out = [ 0.94  0.95], e = 0.0809375650226
x = [0 1], 